In [1]:
!pip install order_of_magnitude
!pip install netCDF4

  Preparing metadata (setup.py) ... done
  Created wheel for order_of_magnitude: filename=order_of_magnitude-2.3.1-py3-none-any.whl size=4581 sha256=862e9afa3a108d41263718b8858f42ac5411badfbb27bcae47825d7aa3c30604
  Stored in directory: /root/.cache/pip/wheels/85/53/9b/310924c3bd35596e70e6d3597f347986a2f95cfb6393e374ec
Successfully built order_of_magnitude
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.2 MB/s eta 0:00:00


In [2]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("drive/My Drive/ERA5_Dataset")

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import netCDF4 as nc
import util

## Target Dataframe

In [4]:
df_target = pd.read_csv('./csv/target_NoOutliers_complete.csv')

lats = df_target['LAT'].values
lons = df_target['LON'].values

df_target

,DATE,ID,LAT,LON,S.IndAll
0,1980-01-01,NaN,-18.50,80.50,0.00
1,1980-01-02,NaN,-30.00,31.25,0.00
2,1980-01-03,NaN,-4.00,86.50,0.00
3,1980-01-04,NaN,-29.50,32.75,0.00
4,1980-01-05,NaN,-4.75,89.25,0.00
...,...,...,...,...,...
15422,2022-03-23,392.0,-13.75,76.75,0.50
15423,2022-03-24,392.0,-14.25,74.25,1.00
15424,2022-03-25,392.0,-13.50,73.25,1.00
15425,2022-03-26,392.0,-15.00,74.00,1.00


## Wind Speed

In [5]:
ds_wind = nc.Dataset('../ERA5_Dataset/nc_original_datasets/wind_gust_10m.nc')

In [6]:
ds_wind['fg10']

<class 'netCDF4._netCDF4.Variable'>
int16 fg10(time, expver, latitude, longitude)
    scale_factor: 0.0008596128197767587
    add_offset: 28.434308382572123
    _FillValue: -32767
    missing_value: -32767
    units: m s**-1
    long_name: 10 metre wind gust since previous post-processing
unlimited dimensions: 
current shape = (15991, 2, 121, 241)
filling on

In [7]:
wind_gust = ds_wind['fg10'][:, 0, :, :]

In [8]:
N_dates = df_target.shape[0]
print(N_dates)

15427


In [9]:
latitude_arr = ds_wind['latitude'][:]
longitude_arr = ds_wind['longitude'][:]

print(latitude_arr.shape, longitude_arr.shape)

(121,) (241,)


In [10]:
print((latitude_arr[0], latitude_arr[30]), (longitude_arr[0], longitude_arr[60]))
print((latitude_arr[0], latitude_arr[30]), (longitude_arr[60], longitude_arr[120]))
print((latitude_arr[0], latitude_arr[30]), (longitude_arr[120], longitude_arr[180]))
print((latitude_arr[0], latitude_arr[30]), (longitude_arr[180], longitude_arr[240]))
print('-------------------------------------------------------')
print((latitude_arr[30], latitude_arr[60]), (longitude_arr[0], longitude_arr[60]))
print((latitude_arr[30], latitude_arr[60]), (longitude_arr[60], longitude_arr[120]))
print((latitude_arr[30], latitude_arr[60]), (longitude_arr[120], longitude_arr[180]))
print((latitude_arr[30], latitude_arr[60]), (longitude_arr[180], longitude_arr[240]))
print('-------------------------------------------------------')
print((latitude_arr[60], latitude_arr[90]), (longitude_arr[0], longitude_arr[60]))
print((latitude_arr[60], latitude_arr[90]), (longitude_arr[60], longitude_arr[120]))
print((latitude_arr[60], latitude_arr[90]), (longitude_arr[120], longitude_arr[180]))
print((latitude_arr[60], latitude_arr[90]), (longitude_arr[180], longitude_arr[240]))
print('-------------------------------------------------------')
print((latitude_arr[90], latitude_arr[120]), (longitude_arr[0], longitude_arr[60]))
print((latitude_arr[90], latitude_arr[120]), (longitude_arr[60], longitude_arr[120]))
print((latitude_arr[90], latitude_arr[120]), (longitude_arr[120], longitude_arr[180]))
print((latitude_arr[90], latitude_arr[120]), (longitude_arr[180], longitude_arr[240]))

(0.0, -7.5) (30.0, 45.0)
(0.0, -7.5) (45.0, 60.0)
(0.0, -7.5) (60.0, 75.0)
(0.0, -7.5) (75.0, 90.0)
-------------------------------------------------------
(-7.5, -15.0) (30.0, 45.0)
(-7.5, -15.0) (45.0, 60.0)
(-7.5, -15.0) (60.0, 75.0)
(-7.5, -15.0) (75.0, 90.0)
-------------------------------------------------------
(-15.0, -22.5) (30.0, 45.0)
(-15.0, -22.5) (45.0, 60.0)
(-15.0, -22.5) (60.0, 75.0)
(-15.0, -22.5) (75.0, 90.0)
-------------------------------------------------------
(-22.5, -30.0) (30.0, 45.0)
(-22.5, -30.0) (45.0, 60.0)
(-22.5, -30.0) (60.0, 75.0)
(-22.5, -30.0) (75.0, 90.0)


In [11]:
print(wind_gust.shape)

wind_zones = []
wind_zones.append(wind_gust[:N_dates, 0:30, 0:60])
wind_zones.append(wind_gust[:N_dates, 0:30, 60:120])
wind_zones.append(wind_gust[:N_dates, 0:30, 120:180])
wind_zones.append(wind_gust[:N_dates, 0:30, 180:])

wind_zones.append(wind_gust[:N_dates, 30:60, 0:60])
wind_zones.append(wind_gust[:N_dates, 30:60, 60:120])
wind_zones.append(wind_gust[:N_dates, 30:60, 120:180])
wind_zones.append(wind_gust[:N_dates, 30:60, 180:])

wind_zones.append(wind_gust[:N_dates, 60:90, 0:60])
wind_zones.append(wind_gust[:N_dates, 60:90, 60:120])
wind_zones.append(wind_gust[:N_dates, 60:90, 120:180])
wind_zones.append(wind_gust[:N_dates, 60:90, 180:])

wind_zones.append(wind_gust[:N_dates, 90:, 0:60])
wind_zones.append(wind_gust[:N_dates, 90:, 60:120])
wind_zones.append(wind_gust[:N_dates, 90:, 120:180])
wind_zones.append(wind_gust[:N_dates, 90:, 180:])

print(len(wind_zones))

(15991, 121, 241)
16


In [12]:
mean_zones = []
std_zones = []

for zone in range(16):
  means = []
  stds = []
  for date_idx in range(N_dates):
    temp_mean = np.mean(wind_zones[zone][date_idx])
    temp_std = np.std(wind_zones[zone][date_idx])
    means.append(temp_mean)
    stds.append(temp_std)
  mean_zones.append(means)
  std_zones.append(stds)

In [14]:
mean_std_zones = np.column_stack((mean_zones[0], std_zones[0],
                                  mean_zones[1], std_zones[1],
                                  mean_zones[2], std_zones[2],
                                  mean_zones[3], std_zones[3],
                                  mean_zones[4], std_zones[4],
                                  mean_zones[5], std_zones[5],
                                  mean_zones[6], std_zones[6],
                                  mean_zones[7], std_zones[7],
                                  mean_zones[8], std_zones[8],
                                  mean_zones[9], std_zones[9],
                                  mean_zones[10], std_zones[10],
                                  mean_zones[11], std_zones[11],
                                  mean_zones[12], std_zones[12],
                                  mean_zones[13], std_zones[13],
                                  mean_zones[14], std_zones[14],
                                  mean_zones[15], std_zones[15]))

# Save Datset

In [15]:
target_att = ['DATE', 'S.IndAll']
mean_std_zones_att = ['Mean_Z1', 'Std_Z1',
                      'Mean_Z2', 'Std_Z2',
                      'Mean_Z3', 'Std_Z3',
                      'Mean_Z4', 'Std_Z4',
                      'Mean_Z5', 'Std_Z5',
                      'Mean_Z6', 'Std_Z6',
                      'Mean_Z7', 'Std_Z7',
                      'Mean_Z8', 'Std_Z8',
                      'Mean_Z9', 'Std_Z9',
                      'Mean_Z10', 'Std_Z10',
                      'Mean_Z11', 'Std_Z11',
                      'Mean_Z12', 'Std_Z12',
                      'Mean_Z13', 'Std_Z13',
                      'Mean_Z14', 'Std_Z14',
                      'Mean_Z15', 'Std_Z15',
                      'Mean_Z16', 'Std_Z16']

target = df_target[target_att].values
final_array = np.hstack((target, mean_std_zones))

df_complete = pd.DataFrame(final_array, columns=target_att+mean_std_zones_att)
df_complete[11:]

,DATE,S.IndAll,Mean_Z1,Std_Z1,Mean_Z2,Std_Z2,Mean_Z3,Std_Z3,Mean_Z4,Std_Z4,...,Mean_Z12,Std_Z12,Mean_Z13,Std_Z13,Mean_Z14,Std_Z14,Mean_Z15,Std_Z15,Mean_Z16,Std_Z16
11,1980-01-12,0.0,5.381555,2.957078,9.447551,0.923882,7.961275,1.479708,8.514588,1.989742,...,9.689114,1.324147,8.407108,3.729831,11.876979,3.282433,13.182545,1.276258,11.520491,1.524789
12,1980-01-13,0.0,5.792269,3.343533,8.198696,1.202385,5.636487,1.357606,7.839208,2.62307,...,11.094159,1.241664,6.894467,2.402398,10.570286,2.881861,12.959018,1.429896,10.963511,1.531548
13,1980-01-14,0.0,5.127522,2.685498,6.858602,1.335227,6.426326,1.596274,8.731751,1.619159,...,11.997676,1.231315,7.366055,2.845647,7.963233,2.26395,12.529026,1.183744,10.867162,1.787463
14,1980-01-15,0.0,5.498013,3.062166,7.926597,1.017251,7.692457,1.668633,9.441629,1.817444,...,11.671708,1.156464,8.374872,3.249949,8.049296,2.693305,10.785515,1.349183,9.882605,2.530827
15,1980-01-16,0.0,5.236773,2.283007,6.309575,1.042154,6.783029,1.340965,8.846063,2.123076,...,10.487875,1.412582,10.438202,4.047074,12.249752,3.266066,10.106409,1.525633,8.857058,1.875648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15422,2022-03-23,0.5,5.072216,1.884,5.048309,1.712216,7.38164,1.551679,8.071348,3.420426,...,14.312355,1.582988,8.614813,3.626762,8.71565,2.121962,8.135213,1.680553,11.752892,1.569658
15423,2022-03-24,1.0,5.24876,2.38554,6.139183,1.419058,10.720927,1.875995,7.240363,3.294448,...,13.185038,2.138392,6.638721,3.962107,9.263336,2.285621,8.099617,1.31113,9.69226,1.969716
15424,2022-03-25,1.0,4.138962,1.860309,5.88907,0.987628,11.888296,2.267146,6.964813,3.96144,...,12.121048,2.617466,6.474759,2.780606,8.066953,2.328406,10.361195,1.410952,10.604379,1.974331
15425,2022-03-26,1.0,4.439019,1.611873,5.0004,1.821387,9.889843,1.28529,5.914646,2.897002,...,12.738067,2.349891,6.225199,2.493933,6.232589,2.065457,9.879186,1.740448,12.419699,1.87029


# Rollback 10 days

In [16]:
def attribute_stepped(atts, max_steps):
  att_list = []
  for att in atts:
    for i in range(max_steps):
      att_list.append('{}-{}'.format(att,i))
  return att_list

In [17]:
def mean_std_atts_zones(zones_n):
  new_att_list = []
  for i in range(zones_n):
    new_att_list.append('Mean_Z{}'.format(i+1))
    new_att_list.append('Std_Z{}'.format(i+1))
  return new_att_list

print(mean_std_atts_zones(16))

['Mean_Z1', 'Std_Z1', 'Mean_Z2', 'Std_Z2', 'Mean_Z3', 'Std_Z3', 'Mean_Z4', 'Std_Z4', 'Mean_Z5', 'Std_Z5', 'Mean_Z6', 'Std_Z6', 'Mean_Z7', 'Std_Z7', 'Mean_Z8', 'Std_Z8', 'Mean_Z9', 'Std_Z9', 'Mean_Z10', 'Std_Z10', 'Mean_Z11', 'Std_Z11', 'Mean_Z12', 'Std_Z12', 'Mean_Z13', 'Std_Z13', 'Mean_Z14', 'Std_Z14', 'Mean_Z15', 'Std_Z15', 'Mean_Z16', 'Std_Z16']


In [18]:
max_steps = 11
labels_to_rewind = mean_std_atts_zones(16)
labels_stepped = attribute_stepped(labels_to_rewind, max_steps)
print(labels_stepped)

stepped_mean_zones = []
stepped_std_zones = []

for zone in range(1, 17):
  mean_press_10d = np.zeros((N_dates-max_steps, max_steps))
  std_press_10d = np.zeros((N_dates-max_steps, max_steps))

  mean_press = df_complete['Mean_Z{}'.format(zone)]
  std_press = df_complete['Std_Z{}'.format(zone)]

  for day in range(max_steps, N_dates):
    for steps in range(max_steps):
      mean_press_10d[day-max_steps, steps] = mean_press[day-steps]
      std_press_10d[day-max_steps, steps] = std_press[day-steps]
  print(mean_press_10d.shape)
  stepped_mean_zones.append(mean_press_10d)
  stepped_std_zones.append(std_press_10d)

['Mean_Z1-0', 'Mean_Z1-1', 'Mean_Z1-2', 'Mean_Z1-3', 'Mean_Z1-4', 'Mean_Z1-5', 'Mean_Z1-6', 'Mean_Z1-7', 'Mean_Z1-8', 'Mean_Z1-9', 'Mean_Z1-10', 'Std_Z1-0', 'Std_Z1-1', 'Std_Z1-2', 'Std_Z1-3', 'Std_Z1-4', 'Std_Z1-5', 'Std_Z1-6', 'Std_Z1-7', 'Std_Z1-8', 'Std_Z1-9', 'Std_Z1-10', 'Mean_Z2-0', 'Mean_Z2-1', 'Mean_Z2-2', 'Mean_Z2-3', 'Mean_Z2-4', 'Mean_Z2-5', 'Mean_Z2-6', 'Mean_Z2-7', 'Mean_Z2-8', 'Mean_Z2-9', 'Mean_Z2-10', 'Std_Z2-0', 'Std_Z2-1', 'Std_Z2-2', 'Std_Z2-3', 'Std_Z2-4', 'Std_Z2-5', 'Std_Z2-6', 'Std_Z2-7', 'Std_Z2-8', 'Std_Z2-9', 'Std_Z2-10', 'Mean_Z3-0', 'Mean_Z3-1', 'Mean_Z3-2', 'Mean_Z3-3', 'Mean_Z3-4', 'Mean_Z3-5', 'Mean_Z3-6', 'Mean_Z3-7', 'Mean_Z3-8', 'Mean_Z3-9', 'Mean_Z3-10', 'Std_Z3-0', 'Std_Z3-1', 'Std_Z3-2', 'Std_Z3-3', 'Std_Z3-4', 'Std_Z3-5', 'Std_Z3-6', 'Std_Z3-7', 'Std_Z3-8', 'Std_Z3-9', 'Std_Z3-10', 'Mean_Z4-0', 'Mean_Z4-1', 'Mean_Z4-2', 'Mean_Z4-3', 'Mean_Z4-4', 'Mean_Z4-5', 'Mean_Z4-6', 'Mean_Z4-7', 'Mean_Z4-8', 'Mean_Z4-9', 'Mean_Z4-10', 'Std_Z4-0', 'Std_Z4-1', 

In [19]:
final_array = np.hstack((target[max_steps:],
                         stepped_mean_zones[0], stepped_std_zones[0],
                         stepped_mean_zones[1], stepped_std_zones[1],
                         stepped_mean_zones[2], stepped_std_zones[2],
                         stepped_mean_zones[3], stepped_std_zones[3],
                         stepped_mean_zones[4], stepped_std_zones[4],
                         stepped_mean_zones[5], stepped_std_zones[5],
                         stepped_mean_zones[6], stepped_std_zones[6],
                         stepped_mean_zones[7], stepped_std_zones[7],
                         stepped_mean_zones[8], stepped_std_zones[8],
                         stepped_mean_zones[9], stepped_std_zones[9],
                         stepped_mean_zones[10], stepped_std_zones[10],
                         stepped_mean_zones[11], stepped_std_zones[11],
                         stepped_mean_zones[12], stepped_std_zones[12],
                         stepped_mean_zones[13], stepped_std_zones[13],
                         stepped_mean_zones[14], stepped_std_zones[14],
                         stepped_mean_zones[15], stepped_std_zones[15],
                         ))
df_stepped = pd.DataFrame(final_array , columns = target_att + labels_stepped)
df_stepped

,DATE,S.IndAll,Mean_Z1-0,Mean_Z1-1,Mean_Z1-2,Mean_Z1-3,Mean_Z1-4,Mean_Z1-5,Mean_Z1-6,Mean_Z1-7,...,Std_Z16-1,Std_Z16-2,Std_Z16-3,Std_Z16-4,Std_Z16-5,Std_Z16-6,Std_Z16-7,Std_Z16-8,Std_Z16-9,Std_Z16-10
0,1980-01-12,0.0,5.381555,4.978972,5.478057,5.856111,5.056396,4.986129,4.749358,5.118156,...,2.42379,2.45997,1.963928,1.503083,3.059897,2.745312,1.846605,1.057159,1.277001,1.453379
1,1980-01-13,0.0,5.792269,5.381555,4.978972,5.478057,5.856111,5.056396,4.986129,4.749358,...,1.524789,2.42379,2.45997,1.963928,1.503083,3.059897,2.745312,1.846605,1.057159,1.277001
2,1980-01-14,0.0,5.127522,5.792269,5.381555,4.978972,5.478057,5.856111,5.056396,4.986129,...,1.531548,1.524789,2.42379,2.45997,1.963928,1.503083,3.059897,2.745312,1.846605,1.057159
3,1980-01-15,0.0,5.498013,5.127522,5.792269,5.381555,4.978972,5.478057,5.856111,5.056396,...,1.787463,1.531548,1.524789,2.42379,2.45997,1.963928,1.503083,3.059897,2.745312,1.846605
4,1980-01-16,0.0,5.236773,5.498013,5.127522,5.792269,5.381555,4.978972,5.478057,5.856111,...,2.530827,1.787463,1.531548,1.524789,2.42379,2.45997,1.963928,1.503083,3.059897,2.745312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15411,2022-03-23,0.5,5.072216,4.910853,4.568998,4.175162,4.037687,4.788805,5.140194,4.937949,...,1.428732,1.774787,0.994377,1.881294,1.533121,1.713125,2.256411,2.028511,2.913761,1.925511
15412,2022-03-24,1.0,5.24876,5.072216,4.910853,4.568998,4.175162,4.037687,4.788805,5.140194,...,1.569658,1.428732,1.774787,0.994377,1.881294,1.533121,1.713125,2.256411,2.028511,2.913761
15413,2022-03-25,1.0,4.138962,5.24876,5.072216,4.910853,4.568998,4.175162,4.037687,4.788805,...,1.969716,1.569658,1.428732,1.774787,0.994377,1.881294,1.533121,1.713125,2.256411,2.028511
15414,2022-03-26,1.0,4.439019,4.138962,5.24876,5.072216,4.910853,4.568998,4.175162,4.037687,...,1.974331,1.969716,1.569658,1.428732,1.774787,0.994377,1.881294,1.533121,1.713125,2.256411


In [20]:
df_stepped.to_csv('./csv/16_zones/wind_gust_10D.csv', index=False)